In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Dữ liệu sử dụng
Thống kê dữ liệu theo từng ngày các trường hợp mắc bệnh, tử vong, hồi phục từ 22/1/2020 - 20/6/2021
<br>
Các dữ liệu đều được lấy từ kho dữ liệu COVID-19 của Trung tâm Khoa học và Kỹ thuật Hệ thống (CSSE) thuộc Đại học Johns Hopkins (USA)
<br>
Đường link lấy dữ liệu: https://github.com/CSSEGISandData/COVID-19
<br>
Điều kiện để được sử dụng dữ liệu: Trích dẫn 1 đoạn mà người public dữ liệu yêu cầu (Đã trích dẫn ở cuối bài làm)
<br>
Người ta thu thập dữ liệu bằng cách lấy thông tin từ 1 số lượng lớn các trang web. Đa số thì sẽ lấy thông tin từ từng Quốc Gia

In [2]:
confirm_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recover_df = pd.read_csv('time_series_covid19_recovered_global.csv')
confirm_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,87716,88740,89861,91458,93272,93288,96531,98734,98734,98734
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132437,132449,132459,132461,132469,132476,132481,132484,132488,132490
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,132727,133070,133388,133742,134115,134458,134840,135219,135586,135821
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,13813,13813,13813,13826,13828,13836,13839,13842,13842,13842
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,36455,36600,36705,36790,36921,37094,37289,37467,37604,37678


# Tiền xử lý lần 1
Gom các cột ngày tháng vào 1 cột có tên là Date, tạo ra 1 bảng duy nhất

In [3]:
dates = confirm_df.iloc[:,4:]
#Không lấy các cột lat và long vì chúng ta không có dự định biểu thị vị trí trên bản đồ thế giới
confirm_df_melt = confirm_df.melt(id_vars=['Province/State', 'Country/Region'], value_vars=dates, var_name='Date', value_name='Confirmed')
death_df_melt = death_df.melt(id_vars=['Province/State', 'Country/Region'], value_vars=dates, var_name='Date', value_name='Deaths')
recover_df_melt = recover_df.melt(id_vars=['Province/State', 'Country/Region'], value_vars=dates, var_name='Date', value_name='Recovered')

In [4]:
#Ghép các giá trị số ca đã xác nhận, số ca tử vong và số ca hồi phục vào dataframe
covid19_df = pd.merge(left=confirm_df_melt, right=death_df_melt, how='left', on=['Province/State', 'Country/Region', 'Date'])
covid19_df = pd.merge(left=covid19_df, right=recover_df_melt, how='left', on=['Province/State', 'Country/Region', 'Date'])
covid19_df.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,1/22/20,0,0,0.0
1,NaN,Albania,1/22/20,0,0,0.0
2,NaN,Algeria,1/22/20,0,0,0.0
3,NaN,Andorra,1/22/20,0,0,0.0
4,NaN,Angola,1/22/20,0,0,0.0


### Số dòng và cột

In [5]:
covid19_df.shape

(143448, 6)

# Chú thích dữ liệu
__Province_State__: Tên tiểu bang, địa danh,...
<br>
__Country_Region__: Tên quốc gia
<br>
__Date__: Thời gian thu thập dữ liệu
<br>
__Confirmed__: Số ca mắc bệnh đã xác nhận
<br>
__Deaths__: Số ca tử vong đã được xác nhận
<br>
__Recovered__: Số ca hồi phục đã được xác nhận

# Khám phá dữ liệu

### Các dòng có lặp không và có bao nhiêu số dòng bị lặp

In [6]:
have_duplicated_rows = covid19_df.duplicated(subset = 'Country/Region').sum()
have_duplicated_rows

143254

### Kiểu dữ liệu

In [7]:
covid19_df.dtypes

Province/State     object
Country/Region     object
Date               object
Confirmed           int64
Deaths              int64
Recovered         float64
dtype: object

### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào ?(Date,Confirmed,Deaths,Recovered)

In [8]:
def missing_ratio(s):
    return s.isna().mean() * 100
covid19_nume = covid19_df.drop(['Province/State','Country/Region'],axis = 1)
nume_col_profiles_df = covid19_df.agg([missing_ratio,min,max])
nume_col_profiles_df

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered
missing_ratio,68.705036,0.0,0.0,0.0,0.0,5.755396e+00
min,NaN,Afghanistan,1/1/21,0.0,0.0,0.000000e+00
max,NaN,Zimbabwe,9/9/20,33541887.0,601824.0,2.884420e+07


### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?(Province/State,Country/Region)

In [11]:
def missing_ratio(s):
    return (s.isna().mean() * 100).round(3)
def num_diff_vals(s):
    return len(s.dropna().unique())
def diff_vals(s):
    return s.dropna().unique()
covid19_cate = covid19_df.select_dtypes(include = 'object').drop(['Date'],axis = 1)
cate_col_profiles_df = covid19_cate.agg([missing_ratio,num_diff_vals,diff_vals])
pd.set_option('display.max_colwidth',None)
cate_col_profiles_df

,Province/State,Country/Region
missing_ratio,68.705,0.0
num_diff_vals,87,194
diff_vals,"[Australian Capital Territory, New South Wales, Northern Territory, Queensland, South Australia, Tasmania, Victoria, Western Australia, Alberta, British Columbia, Diamond Princess, Grand Princess, Manitoba, New Brunswick, Newfoundland and Labrador, Northwest Territories, Nova Scotia, Nunavut, Ontario, Prince Edward Island, Quebec, Repatriated Travellers, Saskatchewan, Yukon, Anhui, Beijing, Chongqing, Fujian, Gansu, Guangdong, Guangxi, Guizhou, Hainan, Hebei, Heilongjiang, Henan, Hong Kong, Hubei, Hunan, Inner Mongolia, Jiangsu, Jiangxi, Jilin, Liaoning, Macau, Ningxia, Qinghai, Shaanxi, Shandong, Shanghai, Shanxi, Sichuan, Tianjin, Tibet, Unknown, Xinjiang, Yunnan, Zhejiang, Faroe Islands, Greenland, French Guiana, French Polynesia, Guadeloupe, Martinique, Mayotte, New Caledonia, Reunion, Saint Barthelemy, Saint Pierre and Miquelon, St Martin, Wallis and Futuna, Aruba, Bonaire, Sint Eustatius and Saba, Curacao, Sint Maarten, Cook Islands, Anguilla, Bermuda, British Virgin Islands, Cayman Islands, Channel Islands, Falkland Islands (Malvinas), Gibraltar, Isle of Man, Montserrat, Saint Helena, Ascension and Tristan da Cunha, Turks and Caicos Islands]","[Afghanistan, Albania, Algeria, Andorra, Angola, Antigua and Barbuda, Argentina, Armenia, Australia, Austria, Azerbaijan, Bahamas, Bahrain, Bangladesh, Barbados, Belarus, Belgium, Belize, Benin, Bhutan, Bolivia, Bosnia and Herzegovina, Botswana, Brazil, Brunei, Bulgaria, Burkina Faso, Burma, Burundi, Cabo Verde, Cambodia, Cameroon, Canada, Central African Republic, Chad, Chile, China, Colombia, Comoros, Congo (Brazzaville), Congo (Kinshasa), Costa Rica, Cote d'Ivoire, Croatia, Cuba, Cyprus, Czechia, Denmark, Diamond Princess, Djibouti, Dominica, Dominican Republic, Ecuador, Egypt, El Salvador, Equatorial Guinea, Eritrea, Estonia, Eswatini, Ethiopia, Fiji, Finland, France, Gabon, Gambia, Georgia, Germany, Ghana, Greece, Grenada, Guatemala, Guinea, Guinea-Bissau, Guyana, Haiti, Holy See, Honduras, Hungary, Iceland, India, Indonesia, Iran, Iraq, Ireland, Israel, Italy, Jamaica, Japan, Jordan, Kazakhstan, Kenya, Kiribati, Korea, South, Kosovo, Kuwait, Kyrgyzstan, Laos, Latvia, Lebanon, Lesotho, ...]"


---------Câu hỏi thứ 1

----------Câu hỏi thứ 2

Dong E, Du H, Gardner L. An interactive web-based dashboard to track COVID-19 in real time. Lancet Inf Dis. 20(5):533-534. doi: 10.1016/S1473-3099(20)30120-1